 Задачи к курсу на Stepic "Нейронные сети и обработка текста"
 https://stepik.org/course/54098


In [2]:
#2.1 Векторная модель текста и TF-IDF. Exercise 2
#Закон Ципфа

N=1000
s=2
Z=sum([i**(-s) for i in range(1,N+1)])
print(*[round(1/(rank**s*Z),4) for rank in range(1,N+1)][:35])
ver=[round(1/(rank**s*Z),5) for rank in range(1,N+1)]
ver10 = list(filter(lambda x: x>0.001, ver))
print(1000-len(ver10))
print(sum([1/(rank**s*Z) for rank in range(1,N+1)]))


0.6083 0.1521 0.0676 0.038 0.0243 0.0169 0.0124 0.0095 0.0075 0.0061 0.005 0.0042 0.0036 0.0031 0.0027 0.0024 0.0021 0.0019 0.0017 0.0015 0.0014 0.0013 0.0011 0.0011 0.001 0.0009 0.0008 0.0008 0.0007 0.0007 0.0006 0.0006 0.0006 0.0005 0.0005
976
0.9999999999999989


In [46]:
import math
q=0.5
b1=1*(1-q)/(1-q**1000);
print(b1)
ver10=10/10000
print(ver10)
m= math.log(ver10/b1)/math.log(q)
print(m)
print([b1*q**(n-1) for n in range(1,30)])

0.5
0.001
8.965784284662087
[0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625, 0.001953125, 0.0009765625, 0.00048828125, 0.000244140625, 0.0001220703125, 6.103515625e-05, 3.0517578125e-05, 1.52587890625e-05, 7.62939453125e-06, 3.814697265625e-06, 1.9073486328125e-06, 9.5367431640625e-07, 4.76837158203125e-07, 2.384185791015625e-07, 1.1920928955078125e-07, 5.960464477539063e-08, 2.9802322387695312e-08, 1.4901161193847656e-08, 7.450580596923828e-09, 3.725290298461914e-09, 1.862645149230957e-09]


In [4]:
import os
print(os.getcwd())
os.chdir(r'C:\Users\gy\Documents\GitHub\stepik-dl-nlp')
print(os.getcwd())
import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
    vectorize_texts, SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

C:\Users\gy
C:\Users\gy\Documents\GitHub\stepik-dl-nlp


In [5]:
#2.4 Семинар: классификация новостных текстов. Exercise 2
#Постройте словарь (отображение из строкового представления токенов в их номера) и вектор весов (DF).

ttt=['Казнить нельзя, помиловать. Нельзя наказывать.','Казнить, нельзя помиловать. Нельзя освободить.',
'Нельзя не помиловать.','Обязательно освободить.' ]
tokens= tokenize_corpus(ttt, min_token_size=1)
#print(tokens)
vocabulary, word_doc_freq = build_vocabulary(tokens, min_count=0)
#print(vocabulary)


vocab2=list(zip(vocabulary.keys(), word_doc_freq))
vocab2=sorted(vocab2, key=lambda word: (word[1], word[0]) )
print('\n','********* Result ********','\n',*[voc[0] for voc in vocab2])
print(*[voc[1] for voc in vocab2])


 ********* Result ******** 
 наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


In [66]:
import scipy
import numpy as np


def vectorize_texts_plus(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True, debug_print=False):
    assert mode in {'tfidf2', 'tfidf', 'idf', 'tf', 'bin'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix(
        (len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        # разделить каждую строку на её длину
        result = result.multiply(1 / result.sum(1))
        # разделить каждый столбец на вес слова
        result = result.multiply(1 / word2freq)

    elif mode == "tfidf2":
        result = result.tocsr()
        # разделить каждую строку на её длину
        result = result.multiply(1 / result.sum(1))  
        if debug_print:
            print('TF')
            print(result)
        result = result + np.ones((len(tokenized_texts), len(word2id)))
        if debug_print:
            print('TF+1')
            print(result)
        
        result = scipy.sparse.dok_matrix(np.log(result))
        if debug_print:
            print('log(TF+1)')
            print(result)

            # разделить каждый столбец на вес слова

            print('разделить каждый столбец на вес слова')
        result = result.multiply(1 / word2freq)
        if debug_print:
            print(result)
            print('*********')

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()

In [70]:
#2.4 Семинар: классификация новостных текстов. Exercise 3
#Постройте матрицу признаков для текстов с шага 5 с использованием словаря и вектора весов, полученного на шаге 5.


VECTORIZATION_MODE = 'tfidf2'
vect = vectorize_texts_plus(tokens, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE, scale=False)
print(vect)
nump_vect = vect + np.zeros((len(tokens), len(vocabulary)))
#print(nump_vect)
print(len(nump_vect))
for i in range(len(nump_vect)):
    print('!!!',nump_vect[i])
    np.std(nump_vect[i], 0, ddof=1)
nump_vect_std = nump_vect.std(0, ddof=1)
print('*****')
print(nump_vect)
print('*****')

  (0, 0)	0.24309541961478315
  (0, 1)	0.4486296678751223
  (0, 2)	0.36464311855496295
  (0, 4)	0.7292862371099259
  (1, 0)	0.24309541961478315
  (1, 1)	0.4486296678751223
  (1, 2)	0.36464311855496295
  (1, 3)	0.36464311855496295
  (2, 0)	0.3835761179679408
  (2, 1)	0.3835761179679408
  (2, 5)	1.150728319609446
  (3, 3)	0.8109302162163288
  (3, 6)	1.6218604324326575
4
!!! [[0.24309542 0.44862967 0.36464312 0.         0.72928624 0.
  0.        ]]
!!! [[0.24309542 0.44862967 0.36464312 0.36464312 0.         0.
  0.        ]]
!!! [[0.38357612 0.38357612 0.         0.         0.         1.15072832
  0.        ]]
!!! [[0.         0.         0.         0.81093022 0.         0.
  1.62186043]]
*****
[[0.24309542 0.44862967 0.36464312 0.         0.72928624 0.
  0.        ]
 [0.24309542 0.44862967 0.36464312 0.36464312 0.         0.
  0.        ]
 [0.38357612 0.38357612 0.         0.         0.         1.15072832
  0.        ]
 [0.         0.         0.         0.81093022 0.         0.
  1.621860

D:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
D:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:130: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
MAX_DF=0.8
MIN_COUNT=0

sc_vector=TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=MAX_DF,
                                                      min_df=MIN_COUNT, norm=None, sublinear_tf=True, smooth_idf=True)
sc_vect_val=sc_vector.fit_transform(ttt)
print(sc_vect_val)
print(sc_vector)

  (0, 0)	1.5108256237659907
  (0, 2)	2.0709620553277333
  (0, 5)	1.2231435513142097
  (0, 1)	1.916290731874155
  (1, 0)	1.5108256237659907
  (1, 2)	2.0709620553277333
  (1, 5)	1.2231435513142097
  (1, 4)	1.5108256237659907
  (2, 2)	1.2231435513142097
  (2, 5)	1.2231435513142097
  (3, 4)	1.5108256237659907
  (3, 3)	1.916290731874155
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=0, ngram_range=(1, 1), norm=None, preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize_text_simple_regex at 0x0000025F09EFA268>,
                use_idf=True, vocabulary=None)


In [41]:
feature_matrix = np.zeros((len(tokens), len(vocabulary)))
feats_std = feature_matrix.std(0, ddof=1)
print(feats_std)

[0. 0. 0. 0. 0. 0. 0.]


In [45]:
# 2.4 Семинар: классификация новостных текстов Exercise 5
# цените количество памяти, которое экономится при использовании разреженной матрицы

doc_n=100000
dict_n=10000
k_fill=0.5/100

full_mb=doc_n*dict_n*4/(1024**2)
spare_mb=doc_n*dict_n*12*k_fill/(1024**2)
print(full_mb, spare_mb, full_mb-spare_mb)

3814.697265625 57.220458984375 3757.476806640625


In [ ]:
# 3.6 Свёрточные нейросети для обработки текстов Exercise 2
# Примените свёртку с ядром (−0.5,0,0.5) к сигналу (1,1,2,3,3,3,2,1,1). 

import numpy as np

vect = [1,1,2,3,3,3,2,1,1]
svert = np.array([-0.5,0,0.5])

for i in range(len(vect)-2):
    a=np.array(vect[i:i+3])
    print(np.dot(a,svert), end= ' ')